<h2>Python: Generative Art</h2>
<hr style='background-color:yellow;height:2px'>
<b>Title:</b>
<span style='color:blue; font-size:1.2em'> OpenGL for a rainy day on a cylinder in PyGame.</span>
<br><b>Hint:</b> We use PyOpenGL to have a cylinder covered with a modifying texture in PyGame.
<br><b>Author:</b> <span style='color:red; font-weight:bold;'>Hamed Shah-Hosseini</span>, Twitter:@chekaad, Instagram:@ostad.ai

In [2]:
import pygame as pg
from OpenGL.GL import *
from OpenGL.GLU import *
import random

In [3]:
def loadTexture():
    textureSurface = pg.image.load('images/ezhdeha2.jpg')
    textureData = pg.image.tostring(textureSurface, "RGBA", 1)
    Twidth,Theight = textureSurface.get_size()
    glEnable(GL_TEXTURE_2D)
    texid = glGenTextures(1)
    glBindTexture(GL_TEXTURE_2D, texid)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, Twidth, Theight,
                 0, GL_RGBA, GL_UNSIGNED_BYTE, textureData)
    glGenerateMipmap(GL_TEXTURE_2D);
    glTexParameterf(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameterf(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    glTexParameterf(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_NEAREST)
    glTexParameterf(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_NEAREST)

    return texid,textureSurface

In [4]:
pg.init()
width,height=800,600
screen=pg.display.set_mode((width,height),pg.DOUBLEBUF| pg.OPENGL)
pg.display.set_caption('OpenGL in PyGame for texture mapping on cylinder'+\
                      ', by Hamed Shah-Hosseini, Twitter:@chekaad, Instagram: @ostad.ai')
glcyn=gluNewQuadric()
def setupRC():
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    gluPerspective(45.0, float(width)/float(height), 0.1, 100.0)
    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()
    glClearColor(.3,.2,.1,1.)
    gluQuadricNormals(glcyn, GLU_SMOOTH) 
    gluQuadricTexture(glcyn, GL_TRUE)
    #-----------
    glEnable(GL_DEPTH_TEST)
    glLightfv(GL_LIGHT0, GL_AMBIENT, (0.3, 0.3, 0.3, 1.0)) 
    glLightfv(GL_LIGHT0, GL_DIFFUSE, (1.0, 1.0, 1.0, 1.0)) 
    glLightfv(GL_LIGHT0, GL_POSITION, (0.0, 0.0, 3.0, 1.0)) 
    glLightfv(GL_LIGHT0,GL_SPECULAR,(1.0,1.0,1.0,1.0))
    glEnable(GL_LIGHT0)
    glEnable(GL_LIGHTING)
    glEnable(GL_COLOR_MATERIAL)
    glColorMaterial(GL_FRONT,GL_AMBIENT_AND_DIFFUSE)
    glMaterialfv(GL_FRONT,GL_SPECULAR,(1.0,1.0,1.0,1.0))
    glMateriali(GL_FRONT,GL_SHININESS,100)
    texid,texSurface=loadTexture()
    return texid, texSurface
#---------------------------
speed=.6; dx=9.; dy=13.
N=500; xys=[];colors=[]
def randXY():
    return [1.7*width*random.random(),random.random()*-height/2]
for i in range(N):
    xys.append(randXY())
    colors.append(random.randint(0,100))
    
def drawLines(surface):
    for i in range(N):
        x1,y1=xys[i]
        x2,y2=x1-dx,y1+dy
        pg.draw.line(surface,(colors[i],colors[i],255),
                                [x1,y1],[x2,y2],width=1)
        x1-=speed*dx
        y1+=speed*dy
        if x1<0 or y1>height:
            xys[i]=randXY()
        else:
            xys[i]=[x1,y1]
#----------------
def RenderScene(angle):
    glClear(GL_COLOR_BUFFER_BIT|GL_DEPTH_BUFFER_BIT)
    cynH=5.
    glColor3f(.8,.4,.4)
    glPushMatrix()
    glTranslatef(0.,0.,-10.)
    glTranslate(0.,0.,cynH/2.)
    glRotatef(angle*2,1.0,0.0,0.0)
    glRotatef(angle*4,0.0,1.0,0.0)
    glRotatef(angle*8,0.0,0.0,1.0)
    glTranslate(0.,0.,-cynH/2.)
    gluCylinder(glcyn,1.25,1.25,cynH,32,32)
    glPopMatrix()
_,texSurface=setupRC()
wFlag=True
angle=0.
while wFlag:
    for event in pg.event.get():
        if event.type==pg.QUIT:
            wFlag=False
        elif event.type==pg.KEYDOWN and event.key==pg.K_ESCAPE:
            wFlag=False
    RenderScene(angle)
    angle+=.1
    if angle>=360.:
        angle=-360.
    pg.display.flip()
    pg.time.wait(10)
    #---------------
    texImage=texSurface.copy()
    drawLines(texImage)
    Tw,Th=texImage.get_size()
    texData=pg.image.tostring(texImage, "RGBA", 1)
    glTexSubImage2D(GL_TEXTURE_2D, 0, 0, 0, Tw, Th, GL_RGBA, GL_UNSIGNED_BYTE, texData)   
    #------------
pg.quit()